# Solar data download

In [ ]:
#| default_exp sol_dl

In [ ]:
#| export
import os
import drms
from datetime import datetime, timedelta

In [ ]:
#| export
class hmi_downloader:
    def __init__(self, jsoc_email):
        jsoc_email = 'mgjeon@khu.ac.kr'
        self.client = drms.Client(email=jsoc_email, verbose=True)

    def find_harpnum(self, start_time):
        ar_mapping = self.client.query('hmi.Mharp_720s[][%sZ]' % start_time.isoformat('_', timespec='seconds'),
                                       key=['NOAA_AR', 'HARPNUM'])
        return ar_mapping
    
    def setup_download_dir(self, download_dir, noaa_num, harp_num):
        self.download_dir = os.path.join(download_dir, f'NOAA{noaa_num}_HARP{harp_num}')
        self.harp_num = harp_num
        os.makedirs(self.download_dir, exist_ok=True)

    def setup_time_interval(self, start_time, duration):
        self.start_time = start_time
        self.duration = duration
        duration_hour = eval(duration.replace('h', ''))
        self.end_time = start_time + timedelta(hours=duration_hour)
    
    def download_hmi(self, segments = 'Br, Bp, Bt', series='sharp_cea_720s'):

        ds = 'hmi.%s[%d][%s/%s]{%s}' % \
                (series, self.harp_num, self.start_time.isoformat('_', timespec='seconds'), self.duration, segments)

        hmi_dir = os.path.join(self.download_dir, 'hmi')
        os.makedirs(hmi_dir, exist_ok=True)

        r = self.client.export(ds, protocol='fits')
        r.wait()
        download_result = r.download(hmi_dir)
        return download_result
    
    def download_aia(self, wavelength='171'):
        ds = f'aia.lev1_euv_12s[{self.start_time.isoformat("_", timespec="seconds")} \
            / {(self.end_time - self.start_time).total_seconds()}s@60s][{wavelength}]{{image}}'
        
        aia_dir = os.path.join(self.download_dir, f'aia/{wavelength}')
        os.makedirs(aia_dir, exist_ok=True)

        r = self.client.export(ds, protocol='fits')
        r.wait()
        download_result = r.download(aia_dir)
        return download_result    

In [ ]:
year = 2017
month = 9
day = 4
hour = 0
minute = 00

start_time = datetime(year, month, day, hour, minute)
duration = '0.1h'

In [ ]:
jsoc_email = 'mgjeon@khu.ac.kr'

In [ ]:
download_dir = '/Users/mgjeon/obsd/obsdata/sdo_AR/'

In [ ]:
dl = hmi_downloader(jsoc_email)

In [ ]:
res = dl.find_harpnum(start_time)
res

,NOAA_AR,HARPNUM
0,12673,7115
1,12674,7117
2,0,7118
3,12675,7120
4,0,7121
5,12677,7122
6,12675,7123
7,0,7124
8,0,7125


In [ ]:
res.iloc[0]

NOAA_AR    12673
HARPNUM     7115
Name: 0, dtype: int64

In [ ]:
dl.setup_download_dir(download_dir, 12673, 7115)

In [ ]:
dl.setup_time_interval(start_time, duration)
dl.start_time, dl.end_time

(datetime.datetime(2017, 9, 4, 0, 0), datetime.datetime(2017, 9, 4, 0, 6))

In [ ]:
dl.download_hmi()

Export request pending. [id=JSOC_20230713_1028, status=2]
Waiting for 5 seconds...
    record: hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI]
  filename: hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bp.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bp.fits
    record: hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI]
  filename: hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bt.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bt.fits
    record: hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI]
  filename: hmi.sharp_cea_720s.7115.20170904_000000_TAI.Br.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170904_000000_TAI.Br.fits


,record,url,download
0,hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI],http://jsoc.stanford.edu/SUM39/D1653985094/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI],http://jsoc.stanford.edu/SUM39/D1653985094/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,hmi.sharp_cea_720s[7115][2017.09.04_00:00:00_TAI],http://jsoc.stanford.edu/SUM39/D1653985094/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='94')

Export request pending. [id=JSOC_20230713_1371, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][94]
  filename: aia.lev1_euv_12s.2017-09-04T000001Z.94.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/94/aia.lev1_euv_12s.2017-09-04T000001Z.94.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][94]
  filename: aia.lev1_euv_12s.2017-09-04T000101Z.94.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/94/aia.lev1_euv_12s.2017-09-04T000101Z.94.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][94]
  filename: aia.lev1_euv_12s.2017-09-04T000201Z.94.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/94/aia.lev1_euv_12s.2017-09-04T000201Z.94.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][94]
  filename: aia.lev1_euv_12s.2017-09-04T000301Z.94.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/94/aia.lev1_euv_12s.2017-09

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][94],http://jsoc.stanford.edu/SUM8/D1654018805/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='131')

Export request pending. [id=JSOC_20230713_1374, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][131]
  filename: aia.lev1_euv_12s.2017-09-04T000008Z.131.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/131/aia.lev1_euv_12s.2017-09-04T000008Z.131.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][131]
  filename: aia.lev1_euv_12s.2017-09-04T000108Z.131.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/131/aia.lev1_euv_12s.2017-09-04T000108Z.131.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][131]
  filename: aia.lev1_euv_12s.2017-09-04T000208Z.131.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/131/aia.lev1_euv_12s.2017-09-04T000208Z.131.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][131]
  filename: aia.lev1_euv_12s.2017-09-04T000308Z.131.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/131/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][131],http://jsoc.stanford.edu/SUM8/D1654019171/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='171')

Export request pending. [id=JSOC_20230713_1365, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][171]
  filename: aia.lev1_euv_12s.2017-09-04T000010Z.171.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/171/aia.lev1_euv_12s.2017-09-04T000010Z.171.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][171]
  filename: aia.lev1_euv_12s.2017-09-04T000110Z.171.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/171/aia.lev1_euv_12s.2017-09-04T000110Z.171.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][171]
  filename: aia.lev1_euv_12s.2017-09-04T000210Z.171.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/171/aia.lev1_euv_12s.2017-09-04T000210Z.171.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][171]
  filename: aia.lev1_euv_12s.2017-09-04T000310Z.171.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/171/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][171],http://jsoc.stanford.edu/SUM8/D1654018147/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='193')

Export request pending. [id=JSOC_20230713_1380, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][193]
  filename: aia.lev1_euv_12s.2017-09-04T000006Z.193.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/193/aia.lev1_euv_12s.2017-09-04T000006Z.193.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][193]
  filename: aia.lev1_euv_12s.2017-09-04T000106Z.193.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/193/aia.lev1_euv_12s.2017-09-04T000106Z.193.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][193]
  filename: aia.lev1_euv_12s.2017-09-04T000206Z.193.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/193/aia.lev1_euv_12s.2017-09-04T000206Z.193.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][193]
  filename: aia.lev1_euv_12s.2017-09-04T000306Z.193.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/193/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][193],http://jsoc.stanford.edu/SUM66/D1654019789/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='211')

Export request pending. [id=JSOC_20230713_1382, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][211]
  filename: aia.lev1_euv_12s.2017-09-03T235959Z.211.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/211/aia.lev1_euv_12s.2017-09-03T235959Z.211.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][211]
  filename: aia.lev1_euv_12s.2017-09-04T000059Z.211.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/211/aia.lev1_euv_12s.2017-09-04T000059Z.211.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][211]
  filename: aia.lev1_euv_12s.2017-09-04T000159Z.211.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/211/aia.lev1_euv_12s.2017-09-04T000159Z.211.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][211]
  filename: aia.lev1_euv_12s.2017-09-04T000259Z.211.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/211/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][211],http://jsoc.stanford.edu/SUM66/D1654020129/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='304')

Export request pending. [id=JSOC_20230713_1386, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][304]
  filename: aia.lev1_euv_12s.2017-09-04T000007Z.304.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/304/aia.lev1_euv_12s.2017-09-04T000007Z.304.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][304]
  filename: aia.lev1_euv_12s.2017-09-04T000107Z.304.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/304/aia.lev1_euv_12s.2017-09-04T000107Z.304.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][304]
  filename: aia.lev1_euv_12s.2017-09-04T000207Z.304.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/304/aia.lev1_euv_12s.2017-09-04T000207Z.304.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][304]
  filename: aia.lev1_euv_12s.2017-09-04T000307Z.304.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/304/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][304],http://jsoc.stanford.edu/SUM8/D1654020436/S000...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='335')

Export request pending. [id=JSOC_20230713_1389, status=2]
Waiting for 5 seconds...
    record: aia.lev1_euv_12s[2017-09-03T23:59:59Z][335]
  filename: aia.lev1_euv_12s.2017-09-04T000002Z.335.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/335/aia.lev1_euv_12s.2017-09-04T000002Z.335.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:00:59Z][335]
  filename: aia.lev1_euv_12s.2017-09-04T000102Z.335.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/335/aia.lev1_euv_12s.2017-09-04T000102Z.335.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:01:59Z][335]
  filename: aia.lev1_euv_12s.2017-09-04T000202Z.335.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/335/aia.lev1_euv_12s.2017-09-04T000202Z.335.image_lev1.fits
    record: aia.lev1_euv_12s[2017-09-04T00:02:59Z][335]
  filename: aia.lev1_euv_12s.2017-09-04T000302Z.335.image_lev1.fits
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/335/aia.lev1_

,record,url,download
0,aia.lev1_euv_12s[2017-09-03T23:59:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
1,aia.lev1_euv_12s[2017-09-04T00:00:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
2,aia.lev1_euv_12s[2017-09-04T00:01:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
3,aia.lev1_euv_12s[2017-09-04T00:02:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
4,aia.lev1_euv_12s[2017-09-04T00:03:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
5,aia.lev1_euv_12s[2017-09-04T00:04:59Z][335],http://jsoc.stanford.edu/SUM66/D1654020715/S00...,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='1600')

Export request pending. [id=JSOC_20230713_1391, status=2]
Waiting for 5 seconds...
    record: warning=No FITS files were exported. The requested FITS files no longer exist.
  filename: 
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/1600/.1


,record,url,download
0,warning=No FITS files were exported. The reque...,http://jsoc.stanford.edu/SUM8/D1654021057/S00000/,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...


In [ ]:
dl.download_aia(wavelength='1700')

Export request pending. [id=JSOC_20230713_1391, status=2]
Waiting for 5 seconds...
    record: warning=No FITS files were exported. The requested FITS files no longer exist.
  filename: 
  -> ../../../obsd/obsdata/sdo_AR/NOAA12673_HARP7115/aia/1700/.1


,record,url,download
0,warning=No FITS files were exported. The reque...,http://jsoc.stanford.edu/SUM8/D1654021057/S00000/,/Users/mgjeon/obsd/obsdata/sdo_AR/NOAA12673_HA...
